In [8]:
function separate_terms(input::String, abstract_ops::Vector{String}, total_tokens::Vector{String}, tokens_at_end::Vector{String}, tokens_at_begin::Vector{String})::Tuple{Vector{Vector{String}},Vector{Vector{String}},Vector{Vector{String}}}
    # Prepare output vectors:

    # Sort each list by descending length (to prioritize longer tokens when there is overlap).
    tokens_total_with_idx = collect(enumerate(total_tokens))
    sort!(tokens_total_with_idx, by=x -> -length(x[2]))

    tokens_at_end_with_idx = collect(enumerate(tokens_at_end))
    sort!(tokens_at_end_with_idx, by=x -> -length(x[2]))

    tokens_at_begin_with_idx = collect(enumerate(tokens_at_begin))
    sort!(tokens_at_begin_with_idx, by=x -> -length(x[2]))

    # Preprocess the input: remove underscores and replace spaces with asterisks.
    input = replace(replace(input, "_" => ""), " " => "*")
    # Split the input string by '*' signs.
    terms = split(input, "*")

    output_total::Vector{Tuple{Vector{Vector{String}}, Vector{Vector{String}}, Vector{Vector{String}}}}()
    curr_output_total = [String[] for _ in 1:length(total_tokens)]
    curr_output_at_end = [String[] for _ in 1:length(tokens_at_end)]
    curr_output_at_begin = [String[] for _ in 1:length(tokens_at_begin)]
    for term in terms
        if length(term) == 0
            continue
        end

        # first check if it contains an abstract operato string. 
        # First, check for a match with total_tokens.
        found_total = nothing
        for (orig_idx, token) in tokens_total_with_idx
            if startswith(term, token)
                rem = term[length(token)+1:end]
                # Accept if nothing follows...
                if isempty(rem)
                    found_total = (orig_idx, token, rem)
                    break
                    # ...or if the remainder begins with "^" and the rest is a valid number or with ' and then possibly ^ and a number 
                elseif startswith(rem, "^")
                    num_part = rem[2:end]
                    try
                        parse(Int, num_part)
                        found_total = (orig_idx, token, rem)
                        break
                    catch e
                        error("Not a valid number $rem in $token.")
                    end
                end
            end
        end
        if found_total !== nothing
            (idx, token, cleaned) = found_total
            push!(curr_output_total[idx], cleaned)
            continue  # Process next term.
        end

        found_end = nothing   # For tokens that should appear at the end.
        found_begin = nothing # For tokens that should appear at the beginning.

        # Check if term ends with any token from tokens_at_end.
        for (orig_idx, token) in tokens_at_end_with_idx
            if endswith(term, token)
                found_end = (orig_idx, token)
                break
            end
        end

        # Check if term starts with any token from tokens_at_begin.
        for (orig_idx, token) in tokens_at_begin_with_idx
            if startswith(term, token)
                found_begin = (orig_idx, token)
                break
            end
        end

        # The term must match exactly one type.
        if (found_end === nothing) && (found_begin === nothing)
            error("No matching token found for term: \"$term\"")
        elseif (found_end !== nothing) && (found_begin !== nothing)
            error("Term matches both beginning and ending tokens: \"$term\"")
        end

        if found_end !== nothing
            (idx, token) = found_end
            # Remove the token from the end.
            cleaned = term[1:end-length(token)]
            push!(curroutput_at_end[idx], cleaned)
        else
            (idx, token) = found_begin
            # Remove the token from the beginning.
            cleaned = term[length(token)+1:end]
            push!(curr_output_at_begin[idx], cleaned)
        end
    end

    return output_total, output_at_end, output_at_begin
end


separate_terms (generic function with 1 method)

In [10]:
function expstr_separate(expstr::String)::Tuple{String,Int}
    exp::Int = 1
    if occursin("^", expstr)
        expstr, b = split(expstr, "^")
        exp = parse(Int, b)
    end
    return expstr, exp
end
expstr_separate("'")

("'", 1)

In [ ]:
using LaTeXStrings
using Pkg
Pkg.activate("..")  # Activate the package environment (assumes Project.toml is one level up)
using qAlgebra
using ComplexRationals

In [ ]:
qs = StateSpace("alpha", "beta(t)", "gamma_i", "delta_i", h=QubitPM(), i=(3, QubitPauli()), b=Ladder())

In [ ]:
var_dict, op_dict= base_operators(qs)

In [ ]:
xi, yi, zi = base_operators("i", qs)
xj, yj, zj = base_operators("j", qs)
xk, yk, zk = base_operators("k", qs)
ph, mh, zh = base_operators("h", qs)
b = base_operators("b", qs)
I = base_operators("I", qs)     # Identity operator
alpha, beta, gamma_i, gamma_j, gamma_k, delta_i, delta_j, delta_k = base_operators("vars", qs)

In [ ]:
expr = 2 * alpha * im * xi + alpha * Dag(b) * xi * yi

In [ ]:
qsum = Sum("j",  alpha*yi*yj+Sum("k", beta*alpha^2*xi*xj*xk))

In [ ]:
flat_sum = flatten(qsum)

In [ ]:
# The sum still covers all combinations of indexes j,k
# We can transform it into a neq sum, in which the indexes j and k are distinct. the following function then expands into all possible cases
neq_sum = neq(qsum) # this also flattens the sum

In [ ]:
# A differential equation of expectation values can be constructed via
dzi_dt = d_dt(zi, alpha*expr+qsum)

In [ ]:
clipboard(string(dzi_dt))

In [ ]:
clipboard(latex_string(dzi_dt))